# 0. Imports and Dataset

In [44]:
try:
    import flax
    import optax
    import ml_collections
    import chex
    import tensorflow
    import tensorflow_datasets
    import rebayes
    import dynamax
    import jax_tqdm
    import bayes_opt
except ModuleNotFoundError:
    %pip install flax
    %pip install optax
    %pip install ml_collections
    %pip install chex
    %pip install tensorflow
    %pip install tensorflow_datasets
    %pip install git+https://github.com/probml/rebayes.git
    %pip install dynamax
    %pip install jax-tqdm
    %pip install bayesian-optimization 
    import flax 
    import optax
    import ml_collections
    import chex
    import tensorflow
    import tensorflow_datasets
    import rebayes
    import dynamax
    import jax_tqdm
    import bayes_opt

In [46]:
import gc
from functools import partial
import time
from typing import Callable, Sequence
import collections.abc
import warnings
import copy
from collections import deque

import ml_collections
import tensorflow_datasets as tfds
from tensorflow_probability.substrates.jax.distributions import MultivariateNormalFullCovariance as MVN
from tensorflow_probability.substrates.jax.distributions import MultivariateNormalDiag as MVND
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from flax import linen as nn
from flax.training import train_state
import jax
from jax import jit, vmap, lax, jacfwd
from jax.flatten_util import ravel_pytree
import jax.numpy as jnp
import jax.random as jr
import optax
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from bayes_opt import BayesianOptimization

jax.numpy.set_printoptions(suppress = True, precision=4)
from rebayes.orfit import GeneralizedORFitParams, RebayesORFit
from rebayes.ekf import RebayesEKF
from rebayes.diagonal_inference import DEKFParams


In [3]:
def get_datasets():
    """Load MNIST train and test datasets into memory."""
    ds_builder = tfds.builder('mnist')
    ds_builder.download_and_prepare()
    train_ds = tfds.as_numpy(ds_builder.as_dataset(split='train', batch_size=-1))
    train_tvsplit_ds = tfds.as_numpy(ds_builder.as_dataset(split='train[:80%]', batch_size=-1))
    val_tvsplit_ds = tfds.as_numpy(ds_builder.as_dataset(split='train[80%:]', batch_size=-1))
    test_ds = tfds.as_numpy(ds_builder.as_dataset(split='test', batch_size=-1))
    for ds in [train_ds, train_tvsplit_ds, val_tvsplit_ds, test_ds]:
        ds['image'] = jnp.float32(ds['image']) / 255.
    return train_ds, test_ds, train_tvsplit_ds, val_tvsplit_ds

In [4]:
def get_config(
    optimizer='sgd',
    learning_rate=0.01, 
    momentum=2e-1,
    init_var=3e-2,
    num_iter=1,
    batch_size=1, 
    num_epochs=1,
    sample_freq=500,
    posterior_predictive_method='mc',
    seed=0
    ):
    """Get the default hyperparameter configuration."""
    config = ml_collections.ConfigDict()
    config.optimizer = optimizer
    config.learning_rate = learning_rate
    config.momentum = momentum
    config.init_var = init_var
    config.num_iter = num_iter
    config.batch_size = batch_size
    config.num_epochs = num_epochs
    config.sample_freq = sample_freq
    config.posterior_predictive_method = posterior_predictive_method
    config.seed = seed
    return config

In [6]:
train_ds, test_ds, train_tvsplit_ds, val_tvsplit_ds = get_datasets()

In [7]:
X_train, y_train = jnp.array(train_ds['image']), jnp.array(train_ds['label'])
X_test, y_test = jnp.array(test_ds['image']), jnp.array(test_ds['label'])

In [8]:
# Reshape data
X_train = X_train.reshape(-1, 1, 28, 28, 1)
y_train_ohe = jax.nn.one_hot(y_train, 10)

# 1. Models

In [9]:
class CNN(nn.Module):
    """A simple CNN model."""
    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=32, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = nn.Conv(features=64, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = x.reshape((x.shape[0], -1))  # flatten
        x = nn.Dense(features=128)(x)
        x = nn.relu(x)
        x = nn.Dense(features=10)(x)
        return x

class MLP(nn.Module):
    features: Sequence[int]

    @nn.compact
    def __call__(self, x):
        x = x.ravel()
        for feat in self.features[:-1]:
            x = nn.relu(nn.Dense(feat)(x))
        x = nn.Dense(self.features[-1])(x)
        return x

In [10]:
cnn = CNN()
key = jr.PRNGKey(0)
params = cnn.init(key, jnp.ones([1, 28, 28, 1]))['params']
flat_params, unflatten_fn = ravel_pytree(params)
print(f'Params size = {flat_params.shape}')
state_dim = flat_params.shape[0]

Params size = (421642,)


In [11]:
apply_fn = lambda w, x: cnn.apply({'params': unflatten_fn(w)}, x).ravel()
emission_mean_function=lambda w, x: jax.nn.softmax(apply_fn(w, x))
def emission_cov_function(w, x):
    ps = emission_mean_function(w, x)
    return jnp.diag(ps) - jnp.outer(ps, ps) + 1e-3 * jnp.eye(len(ps)) # Add diagonal to avoid singularity

In [69]:
def evaluate_neg_log_likelihood(flat_params, unflatten_fn, apply_fn, test_set):
    """ Evaluate negative log likelihood for given parameters and test set
    """
    @jit
    def evaluate_nll(label, image):
        image = image.reshape((1, 28, 28, 1))
        logits = apply_fn(flat_params, image)
        return optax.softmax_cross_entropy_with_integer_labels(logits=logits, labels=label)
    nlls = vmap(evaluate_nll, (0, 0))(test_set['label'], test_set['image'])
    return nlls.mean()

# 2. Generalized ORFit

In [20]:
from jax.lax import scan
from jax import jacrev, vmap, jit
from tqdm import trange
from jax_tqdm import scan_tqdm


# Helper functions
_jacrev_2d = lambda f, x: jnp.atleast_2d(jacrev(f)(x))
_stable_division = lambda a, b: jnp.where(b.any(), a / b, jnp.zeros(shape=a.shape))
_normalize = lambda v: jnp.where(v.any(), v / jnp.linalg.norm(v), jnp.zeros(shape=v.shape))
_projection_matrix = lambda a: _stable_division(a.reshape(-1, 1) @ a.reshape(1, -1), a.T @ a)
_form_projection_matrix = lambda A: jnp.eye(A.shape[0]) - vmap(_projection_matrix, 1)(A).sum(axis=0)
_project = lambda a, x: _stable_division(a * (a.T @ x), (a.T @ a))
_project_to_columns = lambda A, x: \
    jnp.where(A.any(), vmap(_project, (1, None))(A, x).sum(axis=0), jnp.zeros(shape=x.shape))


def _generalized_orfit_condition_on(m, U, Sigma, eta, y_cond_mean, y_cond_cov, x, y, sv_threshold):
    """Condition step of the ORFit algorithm.
    Args:
        m (D_hid,): Prior mean.
        U (D_hid, D_mem,): Prior basis.
        Sigma (D_mem,): Prior singular values.
        eta (float): Prior precision.
        y_cond_mean (Callable): Conditional emission mean function.
        y_cond_cov (Callable): Conditional emission covariance function.
        x (D_in,): Control input.
        y (D_obs,): Emission.
        sv_threshold (float): Threshold for singular values.
    Returns:
        m_cond (D_hid,): Posterior mean.
        U_cond (D_hid, D_mem,): Posterior basis.
        Sigma_cond (D_mem,): Posterior singular values.
    """
    m_Y = lambda w: y_cond_mean(w, x)
    Cov_Y = lambda w: y_cond_cov(w, x)
    
    yhat = jnp.atleast_1d(m_Y(m))
    R = jnp.atleast_2d(Cov_Y(m))
    # print(f'R: {R}')
    L = jnp.linalg.cholesky(R)
    # print(f'L: {L}')
    A = jnp.linalg.lstsq(L, jnp.eye(L.shape[0]))[0].T
    H = _jacrev_2d(m_Y, m)
    # print(f'A: {A}')
    W_tilde = jnp.hstack([Sigma * U, (H.T @ A).squeeze()])
    S = eta*jnp.eye(W_tilde.shape[1]) + W_tilde.T @ W_tilde
    # print(f'S: {S}')
    K = (H.T @ A) @ A.T - W_tilde @ (jnp.linalg.pinv(S) @ (W_tilde.T @ ((H.T @ A) @ A.T)))
    # print(f'K: {K}')
    # print('\n')

    m_cond = m + K/eta @ (y - yhat)
    U_tilde = (H.T - U @ (U.T @ H.T)) @ A

    def _update_basis(carry, i):
        U, Sigma = carry
        v = U_tilde[:, i]
        u = _normalize(v)
        U_cond = jnp.where(
            Sigma.min() < u @ v, 
            jnp.where(sv_threshold < u @ v, U.at[:, Sigma.argmin()].set(u), U),
            U
        )
        Sigma_cond = jnp.where(
            Sigma.min() < u @ v,
            jnp.where(sv_threshold < u @ v, Sigma.at[Sigma.argmin()].set(u.T @ v), Sigma),
            Sigma,
        )
        return (U_cond, Sigma_cond), (U_cond, Sigma_cond)

    (U_cond, Sigma_cond), _ = scan(_update_basis, (U, Sigma), jnp.arange(U_tilde.shape[1]))

    return m_cond, U_cond, Sigma_cond

In [21]:
@chex.dataclass
class ORFitBel:
    mean: chex.Array
    basis: chex.Array
    sigma: chex.Array

class RebayesORFit:
    def __init__(
        self,
        orfit_params,
    ):
        self.eta = orfit_params.initial_precision
        self.update_fn = _generalized_orfit_condition_on
        self.gamma = orfit_params.dynamics_decay
        self.q = orfit_params.dynamics_noise
        self.f = orfit_params.emission_mean_function
        self.r = orfit_params.emission_cov_function
        self.mu0 = orfit_params.initial_mean
        self.m = orfit_params.memory_size
        self.sv_threshold = orfit_params.sv_threshold
        self.U0 = jnp.zeros((len(self.mu0), self.m))
        self.Sigma0 = jnp.zeros((self.m,))

    def initialize(self):
        return ORFitBel(mean=self.mu0, basis=self.U0, sigma=self.Sigma0)

    # @partial(jit, static_argnums=(0,))
    def update(self, bel, u, y):
        m, U, Sigma = bel.mean, bel.basis, bel.sigma # prior predictive for hidden state
        m_cond, U_cond, Sigma_cond = self.update_fn(m, U, Sigma, self.eta, self.f, self.r, u, y, self.sv_threshold)
        return ORFitBel(mean=m_cond, basis=U_cond, sigma=Sigma_cond)

    def scan(self, X, Y, callback=None):
        num_timesteps = X.shape[0]
        
        @scan_tqdm(num_timesteps)
        def step(bel, t):
            bel = self.update(bel, X[t], Y[t])
            out = None
            if callback is not None:
                out = callback(bel, t, X[t], Y[t])
            return bel, out

        carry = self.initialize()
        bel, outputs = scan(step, carry, jnp.arange(num_timesteps))
        return bel, outputs

In [31]:
orfit_params = GeneralizedORFitParams(
    initial_mean=flat_params,
    initial_precision=1.0,
    dynamics_decay=0.0,
    dynamics_noise=0.0,
    emission_mean_function=emission_mean_function,
    emission_cov_function=emission_cov_function,
    memory_size=20
)
orfit_estimator = RebayesORFit(orfit_params)

In [32]:
orfit_bel, _ = orfit_estimator.scan(X_train[:1000], y_train_ohe[:1000])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [33]:
orfit_mean = orfit_bel.mean

In [36]:
emission_mean_function(orfit_mean, X_test[1].reshape(1,28,28,1))

DeviceArray([0.0351, 0.1237, 0.0676, 0.327 , 0.0599, 0.1037, 0.0531,
             0.0849, 0.0774, 0.0674], dtype=float32)

In [39]:
emission_mean_function(orfit_mean, X_test[3].reshape(1,28,28,1))

DeviceArray([0.0351, 0.1237, 0.0676, 0.327 , 0.0599, 0.1037, 0.0531,
             0.0849, 0.0774, 0.0674], dtype=float32)

In [37]:
y_test[1]

DeviceArray(0, dtype=int32)

# 3. VDEKF

## 3.1 Hyperparameter Tuning

In [71]:
def opt_fn(init_var):
    vdekf_params = DEKFParams(
        initial_mean=flat_params,
        initial_cov_diag=jnp.ones(flat_params.shape[0]) * init_var,
        dynamics_cov_diag=jnp.ones((flat_params.size)) * 1e-4,
        emission_mean_function=emission_mean_function,
        emission_cov_function=emission_cov_function,
    )
    vdekf_estimator = RebayesEKF(vdekf_params, 'vdekf')
    vdekf_bel, _ = vdekf_estimator.scan(X_train[:2000], y_train_ohe[:2000])
    vdekf_mean = vdekf_bel.mean
    log_likelihood = -evaluate_neg_log_likelihood(vdekf_mean, unflatten_fn, apply_fn, test_ds)

    return log_likelihood

In [72]:
vdekf_optimizer = BayesianOptimization(
    f=opt_fn,
    pbounds={'init_var': (1e-4, 1.0)},
)

In [ ]:
vdekf_optimizer.maximize(
    init_points=200,
    n_iter=200,
)

|   iter    |  target   | init_var  |
-------------------------------------


  0%|          | 0/2000 [00:00<?, ?it/s]

| 1         | -2.311    | 0.6531    |


  0%|          | 0/2000 [00:00<?, ?it/s]

| 2         | -2.311    | 0.9175    |


...

  0%|          | 0/2000 [00:00<?, ?it/s]

| 219       | -2.311    | 0.6432    |


  0%|          | 0/2000 [00:00<?, ?it/s]

| 220       | -0.4956   | 0.5562    |


  0%|          | 0/2000 [00:00<?, ?it/s]

# 4. SGD

## 4.1 Hyperparameter Tuning

In [ ]:
@jit
def compute_loss_and_updates_sgd(state, batch_images, batch_labels):
    def loss_fn(params):
        logits = state.apply_fn(params, batch_images)
        loss = jnp.mean(optax.softmax_cross_entropy_with_integer_labels(logits=logits, labels=batch_labels))
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    return grads

In [ ]:
lr, momentum = 0.1, 0.1
tx = optax.sgd(lr, momentum)
opt_state = train_state.TrainState.create(apply_fn=apply_fn, params=flat_params, tx=tx)

In [ ]:
def train_model(state, X_train, y_train):
    @jit
    def apply_updates(state, updates):
        return state.apply_gradients(grads=updates)

    for i in trange(len(X_train)):
        curr_image = X_train[i].reshape(1,28,28,1)
        curr_label = y_train[i]

        updates = compute_loss_and_updates_sgd(state, curr_image, curr_label)
        state = apply_updates(state, updates)
    
    return state

In [ ]:
def sgd_opt_fn(lr, momentum):
    tx = optax.sgd(lr, momentum)
    opt_state = train_state.TrainState.create(apply_fn=apply_fn, params=flat_params, tx=tx)
    result_state = train_model(opt_state, X_train, y_train)
    mean = result_state.params
    log_likelihood = -evaluate_neg_log_likelihood(mean, unflatten_fn, apply_fn, test_ds)

    return log_likelihood

In [ ]:
sgd_optimizer = BayesianOptimization(
    f=sgd_opt_fn,
    pbounds={'lr': (1e-4, 1.0), 'momentum': (1e-4, 1.0)},
)

In [ ]:
sgd_optimizer.maximize(
    init_points=100,
    n_iter=100,
)